### Dates in the Index
- dates in index is nice

In [4]:
import pandas as pd
import numpy as np
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df=pd.read_csv(url)
city_mpg = df.city08
highway_mpg = df.highway08
make = df.make
url = 'https://github.com/mattharrison/datasets/raw/master/data/alta-noaa-1980-2019.csv'
alta_df = pd.read_csv(url)
dates = pd.to_datetime(alta_df.DATE)



/tmp/ipykernel_27904/3771879567.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(url)


In [6]:
snow = (alta_df.SNOW.rename(dates)
        )
snow

1980-01-01    2.0
1980-01-02    3.0
1980-01-03    1.0
1980-01-04    0.0
1980-01-05    0.0
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

### 13.1 Finding Missing Data
Looking for missing data methods:

- `.any` : checks for missing data

In [7]:
snow.isna().any()

True

In [8]:
#checks for where missing data is:
snow[snow.isna()]

1985-07-30   NaN
1985-09-12   NaN
1985-09-19   NaN
1986-02-07   NaN
1986-06-26   NaN
              ..
2017-04-26   NaN
2017-09-20   NaN
2017-10-02   NaN
2017-12-23   NaN
2018-12-03   NaN
Name: SNOW, Length: 365, dtype: float64

With date index, can provide partial date strings to the `.loc` index attribute.
Allows us to inspect around missing data and see if it gives any insight into with its missing:

In [9]:
snow.loc['1985-09':'1985-09-20']

1985-09-01    0.0
1985-09-02    0.0
1985-09-03    0.0
1985-09-04    0.0
1985-09-05    0.0
1985-09-06    0.0
1985-09-07    0.0
1985-09-08    0.0
1985-09-09    0.0
1985-09-10    0.0
1985-09-11    0.0
1985-09-12    NaN
1985-09-13    0.0
1985-09-14    0.0
1985-09-15    0.0
1985-09-16    0.0
1985-09-17    0.0
1985-09-18    0.0
1985-09-19    NaN
1985-09-20    0.0
Name: SNOW, dtype: float64

### 13.2 Filling in Missing Data
Multiple ways of doing this, can use `.fillna()`

In [10]:
(snow.loc['1985-09':'1985-09-20']
     .fillna(0)
     )

1985-09-01    0.0
1985-09-02    0.0
1985-09-03    0.0
1985-09-04    0.0
1985-09-05    0.0
1985-09-06    0.0
1985-09-07    0.0
1985-09-08    0.0
1985-09-09    0.0
1985-09-10    0.0
1985-09-11    0.0
1985-09-12    0.0
1985-09-13    0.0
1985-09-14    0.0
1985-09-15    0.0
1985-09-16    0.0
1985-09-17    0.0
1985-09-18    0.0
1985-09-19    0.0
1985-09-20    0.0
Name: SNOW, dtype: float64

However, some values in Jan, middle of winter and etc. might not be zero, the best way to do with missing data is the talk to a subject matter expert and determine why it is missing:

In [11]:
snow.loc['1987-12-30':'1988-01-10']

1987-12-30    6.0
1987-12-31    5.0
1988-01-01    NaN
1988-01-02    0.0
1988-01-03    0.0
1988-01-04    NaN
1988-01-05    2.0
1988-01-06    6.0
1988-01-07    4.0
1988-01-08    9.0
1988-01-09    5.0
1988-01-10    2.0
Name: SNOW, dtype: float64

Pandas has multiple tricks for dealing with missing data
- can use foward fill or back fill using `.ffill` and `.bfill` :

In [12]:
snow.loc['1987-12-30':'1988-01-10'].ffill()

1987-12-30    6.0
1987-12-31    5.0
1988-01-01    5.0
1988-01-02    0.0
1988-01-03    0.0
1988-01-04    0.0
1988-01-05    2.0
1988-01-06    6.0
1988-01-07    4.0
1988-01-08    9.0
1988-01-09    5.0
1988-01-10    2.0
Name: SNOW, dtype: float64

In [13]:
snow.loc['1987-12-30':'1988-01-10'].bfill()

1987-12-30    6.0
1987-12-31    5.0
1988-01-01    0.0
1988-01-02    0.0
1988-01-03    0.0
1988-01-04    2.0
1988-01-05    2.0
1988-01-06    6.0
1988-01-07    4.0
1988-01-08    9.0
1988-01-09    5.0
1988-01-10    2.0
Name: SNOW, dtype: float64

### 13.3 Interpolation
Can also use `.interpolate` , which by default does linear interpolation for missing values


In [14]:
snow.loc['1987-12-30':'1988-01-10'].interpolate()

1987-12-30    6.0
1987-12-31    5.0
1988-01-01    2.5
1988-01-02    0.0
1988-01-03    0.0
1988-01-04    1.0
1988-01-05    2.0
1988-01-06    6.0
1988-01-07    4.0
1988-01-08    9.0
1988-01-09    5.0
1988-01-10    2.0
Name: SNOW, dtype: float64

- Can use code below to fill in missing winter values, (if the quarter is 1 or 4) with interpolated values and the other values with zero. 
- (Because the index is datetime, we can access `.dt` attributes on it)
- example of the `.where` method with truth table:

| Winter   |      Snow      | 
|----------|:-------------:|
| True (I) |  True (II) |
| False (III) |   False (IV)   | 
 - when it is winter, we are missing snow values, we will interpolate
    - this is sections I and IV
- when it is not winter and snow values are missing, we fill in with 0
    - sections 3 and 4

The `.where` method keeps values where the first parameter is `True`, so we invert the condition with `~` :


In [15]:
winter = (snow.index.quarter == 1) | (snow.index.quarter == 4)
(snow
    .where(~(winter & snow.isna()), snow.interpolate())
    .where(~(~winter & snow.isna()), 0)
    )

1980-01-01    2.0
1980-01-02    3.0
1980-01-03    1.0
1980-01-04    0.0
1980-01-05    0.0
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

In [16]:
(snow
    .where(~(winter & snow.isna()), snow.interpolate())
    .where(~(~winter & snow.isna()), 0)
    .loc[['1985-09-19', '1988-01-01']]
    )

1985-09-19    0.0
1988-01-01    2.5
Name: SNOW, dtype: float64

In [17]:
(snow.loc['1987-12-30':'1988-01-10'].dropna())

1987-12-30    6.0
1987-12-31    5.0
1988-01-02    0.0
1988-01-03    0.0
1988-01-05    2.0
1988-01-06    6.0
1988-01-07    4.0
1988-01-08    9.0
1988-01-09    5.0
1988-01-10    2.0
Name: SNOW, dtype: float64

In [18]:
snow.shift(1) # forward shift

1980-01-01    NaN
1980-01-02    2.0
1980-01-03    3.0
1980-01-04    1.0
1980-01-05    0.0
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

In [19]:
snow.shift(-1) #backward shift

1980-01-01    3.0
1980-01-02    1.0
1980-01-03    0.0
1980-01-04    0.0
1980-01-05    1.0
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    NaN
Name: SNOW, Length: 14160, dtype: float64

### 13.6 Rolling Average
To calculate the five day moving average, leverage `.shift` and do the following:


In [20]:
(snow
    .add(snow.shift(1))
    .add(snow.shift(2))
    .add(snow.shift(3))
    .add(snow.shift(4))
    .div(5)
)

1980-01-01    NaN
1980-01-02    NaN
1980-01-03    NaN
1980-01-04    NaN
1980-01-05    1.2
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

In [21]:
# or use .rolling:
(snow.rolling(5).mean())

1980-01-01    NaN
1980-01-02    NaN
1980-01-03    NaN
1980-01-04    NaN
1980-01-05    1.2
             ... 
2019-09-03    0.0
2019-09-04    0.0
2019-09-05    0.0
2019-09-06    0.0
2019-09-07    0.0
Name: SNOW, Length: 14160, dtype: float64

### 13.7 Resampling
- with dates as the index, we can use `.resample` method to aggregate valuse at different levels
- at a high level, can group date entries by some interval (yrly, monthly, weekly) and then aggregate the values at that inteberal

In [22]:
(snow.resample('M').max())

1980-01-31    20.0
1980-02-29    25.0
1980-03-31    16.0
1980-04-30    10.0
1980-05-31     9.0
              ... 
2019-05-31     5.1
2019-06-30     0.0
2019-07-31     0.0
2019-08-31     0.0
2019-09-30     0.0
Freq: M, Name: SNOW, Length: 477, dtype: float64

The `'M'` string in the `.resample` call is what pandas calls an *offset alias* . This is a string that specifies a grouping frequency. 
- Using M means group all values by the end of the month

If you look at the index of the result, you will see that each date is the end of the month. If you want to aggregate at the end of every two months, can use `'2M'` as the offset alias:

In [23]:
(snow.resample('2M').max())

1980-01-31    20.0
1980-03-31    25.0
1980-05-31    10.0
1980-07-31     1.0
1980-09-30     0.0
              ... 
2019-01-31    19.0
2019-03-31    20.7
2019-05-31    18.0
2019-07-31     0.0
2019-09-30     0.0
Freq: 2M, Name: SNOW, Length: 239, dtype: float64

If you want to aggregate the maximum value for each ski season, which normally ends in May, use the following:
- the offset alias `'A-MAY'`, indicates that we want an annual grouping (`'A'`), but ending in May of each year

In [25]:
(snow
    .resample('A-MAY')
    .max())


1980-05-31    25.0
1981-05-31    26.0
1982-05-31    34.0
1983-05-31    38.0
1984-05-31    25.0
1985-05-31    22.0
1986-05-31    34.0
1987-05-31    16.0
1988-05-31    23.0
1989-05-31    30.0
1990-05-31    32.0
1991-05-31    28.0
1992-05-31    22.0
1993-05-31    30.0
1994-05-31    36.0
1995-05-31    25.0
1996-05-31    34.0
1997-05-31    22.0
1998-05-31    29.0
1999-05-31    26.0
2000-05-31    23.0
2001-05-31    19.0
2002-05-31    28.0
2003-05-31    14.0
2004-05-31    24.0
2005-05-31    31.0
2006-05-31    27.0
2007-05-31    15.0
2008-05-31    21.0
2009-05-31    23.0
2010-05-31    32.0
2011-05-31    22.0
2012-05-31    18.0
2013-05-31    19.0
2014-05-31    11.0
2015-05-31    25.0
2016-05-31    15.0
2017-05-31    26.0
2018-05-31    21.8
2019-05-31    20.7
2020-05-31     0.0
Freq: A-MAY, Name: SNOW, dtype: float64

### 13.8 Gatthering Aggregate Values (but keeping index)
- instead of performing an aggregation with `.resample` we leverage the `.transform` method.
    - works on aggregation groupsd but returns a series with the original index, mkaing it easy to do things like calculate the percentage of quarterly snowfall that fell in a day.

In [26]:
(snow
    .div(snow
            .resample('Q')
            .transform('sum'))
    .mul(100)
    .fillna(0)
    )

1980-01-01    0.527009
1980-01-02    0.790514
1980-01-03    0.263505
1980-01-04    0.000000
1980-01-05    0.000000
                ...   
2019-09-03    0.000000
2019-09-04    0.000000
2019-09-05    0.000000
2019-09-06    0.000000
2019-09-07    0.000000
Name: SNOW, Length: 14160, dtype: float64

To compute the percentage of a season’s snowfall that fell during each month, we could do the following:

In [27]:
season2017 = snow.loc['2016-10':'2017-05']
(season2017
    .resample('M')
    .sum()
    .div(season2017.sum())
    .mul(100)
    )

2016-10-31     2.153969
2016-11-30     9.772637
2016-12-31    15.715995
2017-01-31    25.468688
2017-02-28    21.041085
2017-03-31     9.274033
2017-04-30    14.738732
2017-05-31     1.834862
Freq: M, Name: SNOW, dtype: float64

### 13.9 Groupby Operations
`.groupby` method that acts as a generic sort of `.resample` 
- used more on df than series

example of creating a fn that will determine ski season by looking at the index with date info. Considers a season to be from Oct to Sept:

In [28]:
def season(idx):
    year = idx.year
    month =idx.month
    return year.where((month < 10), year+1)

In [30]:
(snow 
    .groupby(season)
    .sum().div(12)
    )

1980    38.125000
1981    41.916667
1982    70.208333
1983    67.291667
1984    68.000000
1985    44.666667
1986    61.733333
1987    20.258333
1988    26.208333
1989    35.791667
1990    27.625000
1991    42.058333
1992    28.400000
1993    56.958333
1994    26.750000
1995    53.750000
1996    43.791667
1997    46.966667
1998    48.300000
1999    36.308333
2000    37.750000
2001    39.000000
2002    38.150000
2003    30.450000
2004    42.833333
2005    39.333333
2006    49.550000
2007    26.641667
2008    50.500000
2009    39.733333
2010    32.583333
2011    44.483333
2012    24.458333
2013    30.233333
2014    29.891667
2015    23.691667
2016    29.550000
2017    43.666667
2018    25.733333
2019    42.041667
Name: SNOW, dtype: float64

In [31]:
(snow
    .resample('A-SEP')
    .sum().div(12)
    ) #also works

1980-09-30    38.125000
1981-09-30    41.916667
1982-09-30    70.208333
1983-09-30    67.291667
1984-09-30    68.000000
1985-09-30    44.666667
1986-09-30    61.733333
1987-09-30    20.258333
1988-09-30    26.208333
1989-09-30    35.791667
1990-09-30    27.625000
1991-09-30    42.058333
1992-09-30    28.400000
1993-09-30    56.958333
1994-09-30    26.750000
1995-09-30    53.750000
1996-09-30    43.791667
1997-09-30    46.966667
1998-09-30    48.300000
1999-09-30    36.308333
2000-09-30    37.750000
2001-09-30    39.000000
2002-09-30    38.150000
2003-09-30    30.450000
2004-09-30    42.833333
2005-09-30    39.333333
2006-09-30    49.550000
2007-09-30    26.641667
2008-09-30    50.500000
2009-09-30    39.733333
2010-09-30    32.583333
2011-09-30    44.483333
2012-09-30    24.458333
2013-09-30    30.233333
2014-09-30    29.891667
2015-09-30    23.691667
2016-09-30    29.550000
2017-09-30    43.666667
2018-09-30    25.733333
2019-09-30    42.041667
Freq: A-SEP, Name: SNOW, dtype: float64

### 13.10 Cumulative Operations
- `.cummin`
- `.cummax`
- `.cumprod`
- `.cumsum`

To calculate snowfall in a season, we can combine `.cumsum` with slicing

In [32]:
(snow
    .loc['2016-10':'2017-09']
    .cumsum())

2016-10-01      0.0
2016-10-02      0.0
2016-10-03      4.9
2016-10-04      4.9
2016-10-05      5.5
              ...  
2017-09-26    524.0
2017-09-27    524.0
2017-09-28    524.0
2017-09-29    524.0
2017-09-30    524.0
Name: SNOW, Length: 364, dtype: float64

To do calculation for every year, combine `.resample` with `.transform` and `.cumsum`:

In [33]:
(snow
    .resample('A-SEP')
    .transform('cumsum')
    )

1980-01-01      2.0
1980-01-02      5.0
1980-01-03      6.0
1980-01-04      6.0
1980-01-05      6.0
              ...  
2019-09-03    504.5
2019-09-04    504.5
2019-09-05    504.5
2019-09-06    504.5
2019-09-07    504.5
Name: SNOW, Length: 14160, dtype: float64